In [ ]:
# 1. 랭체인 패밀리 강제 업데이트 및 설치
import os

print("🔄 1. 구글 드라이브 마운트 확인 (필수 데이터 접근용)")
from google.colab import drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
else:
    print("✅ 이미 마운트 되어 있습니다.")


!CMAKE_ARGS="-DGGML_CUDA=on" pip install llama-cpp-python[server] \
    --force-reinstall \
    --no-cache-dir

print("\n 3. 나머지 RAG 필수 라이브러리 일괄 설치")
# 요청하신 리스트 + 의존성 패키지들
!pip install -U \
    langchain \
    langchain-community \
    langchain-core \
    chromadb \
    langchain-text-splitters \
    langchain-openai \
    langgraph \
    langfuse \
    sentence-transformers \
    jq \
    langchain-huggingface \


# 2. 설치가 끝나면, Colab 상단 메뉴의 [런타임] -> [세션 다시 시작]을 꼭 눌러주세요!
# (라이브러리 꼬임 방지)

In [1]:
import llama_cpp
print(f"Llama version: {llama_cpp.__version__}")
# GPU 레이어 오프로드 지원 여부 확인 (True가 나와야 함)
# (간접적인 확인 방법입니다. 실행 로그에 BLAS = 1 이 떠야 진짜입니다.)

Llama version: 0.3.16


In [24]:
import subprocess
import time
import requests
import os

# 모델 경로 (아까 다운로드 받은 경로 확인 필수)
MODEL_PATH = "/content/drive/MyDrive/수능 풀이/models/Qwen3-30B-A3B-Instruct-2507-UD-Q6_K_XL.gguf"

if not os.path.exists(MODEL_PATH):
    print(f"❌ 모델 파일이 없습니다! 경로를 확인하세요: {MODEL_PATH}")
else:
    # 서버 실행 (GPU 풀가동)
    cmd = [
        "python", "-m", "llama_cpp.server",
        "--model", MODEL_PATH,
        "--n_gpu_layers", "-1",  # L4 GPU 사용
        "--n_ctx", "32768",       # 긴 문맥(RAG) 지원
        "--host", "0.0.0.0",
        "--port", "8000",
        "--flash_attn", "true"
    ]

    print(f"🚀 llama.cpp 서버 시작 중... (모델: {os.path.basename(MODEL_PATH)})")
    server_process = subprocess.Popen(cmd, stdout=open("server_log.txt", "w"), stderr=subprocess.STDOUT)

    # Health Check
    print("⏳ 서버 부팅 대기 중...", end="")
    while True:
        try:
            requests.get("http://localhost:8000/v1/models")
            print("\n✅ 서버 준비 완료!")
            break
        except:
            print(".", end="")
            time.sleep(2)

🚀 llama.cpp 서버 시작 중... (모델: Qwen3-30B-A3B-Instruct-2507-UD-Q6_K_XL.gguf)
⏳ 서버 부팅 대기 중................
✅ 서버 준비 완료!


In [6]:
import glob
from langchain_community.document_loaders import JSONLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
# 🔥 중요: 최신 패키지로 변경 (경고 메시지 해결)
from langchain_huggingface import HuggingFaceEmbeddings

# ==========================================
# 2. 데이터 로드 (jq 에러 해결됨)
# ==========================================
print("📚 데이터 로딩 시작...")

# 경로 확인 (본인 경로에 맞게 수정)
DATA_PATH = "/content/drive/MyDrive/수능 풀이/history_data_final/*.json"
files = glob.glob(DATA_PATH)

documents = []

def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["volume"] = record.get("volume")
    metadata["title"] = record.get("title")
    return metadata

# 파일 로드
for file in files:
    try:
        loader = JSONLoader(
            file_path=file,
            jq_schema='.[]',     # 이제 jq가 깔려서 잘 될 겁니다!
            content_key='text',
            metadata_func=metadata_func
        )
        documents.extend(loader.load())
    except Exception as e:
        print(f"❌ {file} 로드 실패: {e}")

print(f"✅ 총 {len(documents)}개의 문서 로드 완료!")

# ==========================================
# 3. 임베딩 & 벡터 DB (다운로드 기다려주세요!)
# ==========================================
if documents:
    # 청킹
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(documents)
    print(f"✂️ {len(splits)}개의 청크로 분할됨.")

    # 임베딩 모델 (여기서 1~2분 걸립니다! 멈추지 마세요!)
    print("⏳ 임베딩 모델 다운로드 및 처리 중... (잠시만 기다려주세요)")
    embedding_model = HuggingFaceEmbeddings(
        model_name="dragonkue/BGE-m3-ko",
        model_kwargs={'device': 'cuda'}
    )

    DB_PATH = "/content/drive/MyDrive/수능 풀이/chroma_db_2"

    print(f"💾 벡터 DB를 '{DB_PATH}'에 저장합니다...")

    # 🔥 중요: persist_directory 옵션 추가!
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=embedding_model,
        collection_name="korean_history_2",
        persist_directory=DB_PATH
    )

    print("✅ 저장 완료! 이제 Colab 꺼져도 데이터는 살아있습니다.")

    # 검색기 생성
    retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
    print("🎉 RAG 준비 끝! 고생하셨습니다.")

else:
    print("⚠️ 로드된 문서가 없습니다. 경로를 다시 확인해주세요.")

📚 데이터 로딩 시작...
✅ 총 8378개의 문서 로드 완료!
✂️ 40671개의 청크로 분할됨.
⏳ 임베딩 모델 다운로드 및 처리 중... (잠시만 기다려주세요)
💾 벡터 DB를 '/content/drive/MyDrive/수능 풀이/chroma_db_2'에 저장합니다...
✅ 저장 완료! 이제 Colab 꺼져도 데이터는 살아있습니다.
🎉 RAG 준비 끝! 고생하셨습니다.


In [ ]:
# from langchain_community.vectorstores import Chroma
# # (임베딩 모델은 로드해야 함 - DB 해석용)
# from langchain_huggingface import HuggingFaceEmbeddings

# # 1. 임베딩 모델 준비
# embedding_model = HuggingFaceEmbeddings(
#     model_name="jhgan/ko-sbert-multitask",
#     model_kwargs={'device': 'cuda'}
# )

# # 2. 저장된 DB 불러오기 (documents 로드 불필요!)
# DB_PATH = "/content/drive/MyDrive/수능 풀이/chroma_db"

# vectorstore = Chroma(
#     persist_directory=DB_PATH,
#     embedding_function=embedding_model,
#     collection_name="korean_history"
# )

# retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
# print("✅ 저장된 DB를 성공적으로 불러왔습니다!")

In [10]:
from langfuse import Langfuse
import os

# 👇 여기에 본인의 키를 '정확하게' 붙여넣으세요! (공백 주의)
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-96924478-61c3-4a5c-a251-dc4ebfe9384c"
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-fe21a007-6fc8-4050-89f3-f6c436845a63"
os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com"



try:
    # 핸들러 말고, '본체(Client)'를 직접 소환합니다.
    langfuse = Langfuse()

    # 정석 인증 체크 함수
    if langfuse.auth_check():
        print("🎉 [최종 확인] 인증 성공! 연결 완벽합니다.")

    else:
        print("⚠️ [인증 실패] 키 값이나 호스트 주소(US/EU)를 다시 봐주세요.")

except Exception as e:
    print(f"🔥 에러: {e}")

🎉 [최종 확인] 인증 성공! 연결 완벽합니다.


In [28]:
langfuse_handler = None
try:
    # ✅ 정답 경로: langfuse.langchain
    from langfuse.langchain import CallbackHandler

    # 인증 체크 함수(auth_check) 호출하지 않고 바로 생성 (에러 방지)
    langfuse_handler = CallbackHandler()
    print("✅ LangFuse 핸들러 연결 성공! (로그 적재 중)")

except ImportError:
    print("⚠️ LangFuse 패키지 경로 에러: 로그 없이 진행합니다.")
except Exception as e:
    print(f"⚠️ LangFuse 연결 실패 ({e}): 로그 없이 진행합니다.")

✅ LangFuse 핸들러 연결 성공! (로그 적재 중)


In [25]:
from langchain_openai import ChatOpenAI

# LLM 재연결 (타임아웃 추가)
base_llm = ChatOpenAI(
    base_url="http://localhost:8000/v1",
    api_key="EMPTY",
    model=MODEL_PATH,
    streaming=False,
    max_tokens=2048,
    request_timeout=600, # 🔥 120초(2분) 지나면 에러 뱉고 다음 문제로 넘어감
    max_retries=1        # 재시도 1회만
)

# 파라미터 다시 바인딩
llm_with_params = base_llm.bind(
    temperature=0.7, # 문제 풀이용
    top_p=0.90,
    extra_body={
        "top_k": 25,
        "repeat_penalty": 1.1
    }
)

# (이후 LangGraph 노드 정의 및 app.compile() 코드는 그대로 사용)
print("✅ LLM 클라이언트 업데이트 완료 (Timeout 적용)")

✅ LLM 클라이언트 업데이트 완료 (Timeout 적용)


In [26]:
import re
from typing import TypedDict, List
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import StateGraph, END

# =========================================================
# 1. State 정의 (데이터가 흘러가는 통로)
# =========================================================
class MCQState(TypedDict):
    id: str
    paragraph: str
    question: str
    choices: List[str]
    retrieved_context: str  # 최종 수집된 증거 자료
    full_response: str      # LLM의 풀이 과정 (Raw)
    final_answer: str       # 추출된 정답 번호

# =========================================================
# 2. 핵심 노드: Retrieve Node (지능형 검색기)
# =========================================================
def retrieve_node(state: MCQState):
    print(f"\n🔎 [Retriever] 문제({state.get('id', 'Unknown')}) 분석 시작...")

    # -----------------------------------------------------
    # Phase 1: [Router] 문제 유형 분류 (3대장 전략)
    # -----------------------------------------------------
    router_prompt = ChatPromptTemplate.from_template(
        """<|im_start|>system
당신은 '문제 해결 전략가'입니다. 주어진 문제를 보고 최적의 검색 전략을 하나 선택하십시오.

[분류 기준]
1. **INFERENCE**: (가), (나), "이 인물", "이 단체" 처럼 주어가 가려져 있어 문맥 묘사를 통해 대상을 찾아야 하는 경우.
2. **SEQUENCE**: "(가)와 (나) 사이", "순서대로 나열", "연표", "시기" 등 시간의 흐름이나 연도를 묻는 경우.
3. **GENERAL**: 위 두 경우에 해당하지 않는 일반적인 사실 확인 문제 (대상이 명확한 경우).

반드시 **INFERENCE**, **SEQUENCE**, **GENERAL** 중 단어 하나만 출력하십시오.
<|im_start|>user
[지문]: {paragraph}
[질문]: {question}
[선지]: {choices}

전략:<|im_end|>
<|im_start|>assistant
"""
    )

    router_chain = router_prompt | llm_with_params | StrOutputParser()
    strategy = router_chain.invoke({
        "paragraph": state['paragraph'],
        "question": state['question'],
        "choices": str(state['choices'])
    }).strip()

    print(f"   🚦 [전략 선택]: {strategy}")

    # -----------------------------------------------------
    # Phase 2: [Query Gen] 전략별 검색 프롬프트 분기
    # -----------------------------------------------------
    if "INFERENCE" in strategy:
        # 전략 A: 몽타주 검색 (이름 추측 금지)
        sys_msg = """당신은 '역사 탐정'입니다. 숨겨진 주어((가), (나))를 찾기 위한 '몽타주 검색어'를 만드십시오.
**주의:** (가)가 누구인지 절대 추측하여 특정 이름(예: 김구, 신라)을 넣지 마십시오. 오직 지문에 묘사된 '행동', '사건 내용', '단체명', '장소'를 나열하십시오."""

    elif "SEQUENCE" in strategy:
        # 전략 B: 연도/타임라인 검색
        sys_msg = """당신은 '연표 분석가'입니다. 사건의 순서나 시기를 파악하기 위한 검색어를 만드십시오.
**주의:** 지문에 나온 사건들의 '발생 연도', '시대적 배경', '왕의 재위 기간' 관련 키워드를 반드시 포함하십시오."""

    else: # GENERAL
        # 전략 C: 핵심 요약 검색
        sys_msg = """당신은 '검색 키워드 요약 전문가'입니다. 질문에 답하기 위한 핵심 키워드를 추출하십시오.
**주의:** 불필요한 조사를 빼고, '인물', '사건', '핵심 용어' 위주로 간결하게 요약하십시오."""

    query_gen_prompt = ChatPromptTemplate.from_template(
        f"""<|im_start|>system
{sys_msg}
<|im_start|>user
[지문]: {{paragraph}}
[질문]: {{question}}

최적 검색어:<|im_end|>
<|im_start|>assistant
"""
    )

    gen_chain = query_gen_prompt | llm_with_params | StrOutputParser()
    optimized_query = gen_chain.invoke({
        "paragraph": state['paragraph'],
        "question": state['question']
    }).strip()

    print(f"   ✨ [생성된 쿼리]: {optimized_query}")

    # -----------------------------------------------------
    # Phase 3: [Dual Search] 정밀 검색 + 안전망 가동
    # -----------------------------------------------------
    # 1. 정밀 검색 (LLM이 만든 쿼리)
    docs_a = retriever.invoke(optimized_query)

    # 2. 안전망 검색 (지문 원본 - 환각 방지용)
    docs_b = retriever.invoke(state['paragraph'])

    # 3. 중복 제거 및 합치기 (정밀 검색 우선)
    combined_docs = docs_a + docs_b
    unique_docs = []
    seen = set()
    for d in combined_docs:
        if d.page_content not in seen:
            unique_docs.append(d)
            seen.add(d.page_content)

    final_para_docs = unique_docs[:5] # 상위 5개만 사용
    para_context = "\n".join([f"- {d.page_content}" for d in final_para_docs])

    # -----------------------------------------------------
    # Phase 4: [Choices Search] 선지별 교차 검증
    # -----------------------------------------------------
    choices_evidence = []
    for idx, choice in enumerate(state['choices']):
        # 검색어 = 최적화된 쿼리 + 선지 내용
        combined_q = f"{optimized_query} {choice}"
        choice_docs = retriever.invoke(combined_q)

        if choice_docs:
            evi = " / ".join([d.page_content for d in choice_docs[:2]]) # 선지당 2개만
        else:
            evi = "관련 정보 없음"
        choices_evidence.append(f"[선지 {idx+1}]: {evi}")

    # -----------------------------------------------------
    # Phase 5: 문맥 조립
    # -----------------------------------------------------
    full_context = f"""
=== [배경 지식 (전략: {strategy})] ===
{para_context}

=== [선지 검증 자료] ===
{chr(10).join(choices_evidence)}
"""
    return {"retrieved_context": full_context}

# =========================================================
# 3. Solver Node (문제 풀이)
# =========================================================
def solver_node(state: MCQState):
    choices_str = "\n".join([f"{i+1}. {c}" for i, c in enumerate(state['choices'])])

    system_prompt = """당신은 논리적인 한국사 전문가입니다.
제공된 <핵심 검증 자료>를 근거로 문제를 해결하십시오.
- 자료에 없는 내용은 추측하지 말고, 자료를 바탕으로 논리적으로 오답을 소거하십시오.
- 마지막 줄에는 반드시 {"정답": "번호"} 형식으로 답을 출력하십시오."""

    prompt = ChatPromptTemplate.from_template(
        """<|im_start|>system
{system_msg}<|im_end|>
<|im_start|>user
<지문>
{paragraph}

<질문>
{question}

<선지>
{choices}

<핵심 검증 자료>
{refined_context}
<|im_end|>
<|im_start|>assistant
"""
    )

    chain = prompt | llm_with_params | StrOutputParser()

    response = chain.invoke({
        "system_msg": system_prompt,
        "paragraph": state['paragraph'],
        "question": state['question'],
        "choices": choices_str,
        "refined_context": state['retrieved_context']
    })

    return {"full_response": response}

# =========================================================
# 4. Parser Node (정답 추출)
# =========================================================
def parser_node(state: MCQState):
    text = state['full_response']
    answer = None
    # JSON 형식 찾기
    match = re.search(r'{"정답":\s*"(\d+)"}', text)
    if match:
        answer = match.group(1)
    else:
        # 못 찾으면 텍스트 내 마지막 숫자 추출 (최후의 수단)
        nums = re.findall(r'\d+', text)
        if nums: answer = nums[-1]

    return {"final_answer": answer}

# =========================================================
# 5. 그래프 조립 (Workflow Construction)
# =========================================================
workflow = StateGraph(MCQState)

workflow.add_node("retrieve", retrieve_node)
workflow.add_node("solve", solver_node)
workflow.add_node("parse", parser_node)

# 흐름 연결
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "solve")
workflow.add_edge("solve", "parse")
workflow.add_edge("parse", END)

# 컴파일
app = workflow.compile()
print("🚀 [RAG] 최종 조립 완료! (Router + Dual Search + Choice Verification)")

🚀 [RAG] 최종 조립 완료! (Router + Dual Search + Choice Verification)


In [29]:
import ast
import pandas as pd
from tqdm import tqdm

# 데이터 로드 (73개 테스트)
df_train = pd.read_csv("/content/drive/MyDrive/수능 풀이/data/train.csv")
test_data = df_train.iloc[:73]

results = []
print("🔥 최종 RAG 모델 평가 시작...")

for idx, row in tqdm(test_data.iterrows(), total=len(test_data)):
    try:
        # 데이터 전처리
        problem = ast.literal_eval(row['problems']) if isinstance(row['problems'], str) else row['problems']

        inputs = {
            "id": row.get('id', f"q-{idx}"),
            "paragraph": row['paragraph'],
            "question": problem['question'],
            "choices": problem['choices']
        }

        # LangFuse 콜백 설정
        config = {"callbacks": [langfuse_handler]} if 'langfuse_handler' in globals() else {}

        # 그래프 실행
        output = app.invoke(inputs, config=config)

        # 결과 저장
        results.append({
            "id": inputs['id'],
            "question": inputs['question'],
            "predicted_answer": output['final_answer'],
            "real_answer": str(problem.get('answer')),
            "is_correct": str(output['final_answer']) == str(problem.get('answer')),
            "retrieved_context": output['retrieved_context'], # 검색된 내용 저장
            "full_response": output['full_response']
        })

    except Exception as e:
        print(f"❌ Error at {idx}: {e}")

# 결과 저장 및 출력
final_df = pd.DataFrame(results)
final_df.to_csv("/content/drive/MyDrive/수능 풀이/rag_results.csv", index=False, encoding='utf-8-sig')

acc = final_df['is_correct'].mean() * 100
print(f"\n🏆 최종 정답률: {acc:.2f}%")

🔥 최종 RAG 모델 평가 시작...


  0%|          | 0/73 [00:00<?, ?it/s]


🔎 [Retriever] 문제(generation-for-nlp-425) 분석 시작...
   🚦 [전략 선택]: INFERENCE
   ✨ [생성된 쿼리]: 역사 탐정의 몽타주 검색어:

상소 내용: 상복 3년 제도에 대한 논의, 장자로서의 정체성보다 전 중(조상 제사 및 법통 계승)이 핵심, 할아버지와 아버지의 뒤를 이은 '정체'가 중요함, 첫째 아들이 아니어도 3년 복 가능

연관된 역사적 맥락: 현종실록 기록, 정권 장악(사환국), 인조반정 주도, 정조 시대 탕평 정치의 한 축, 이이와 성혼 문인 중심 세력 형성

→ 몽타주 검색어:

현종 실록 상소문, 상복 3년 복 제도 논의, 전 중(傳重) 핵심 주장, 정체성 기반 장자 의무 부정, 첫째 아들 아님에도 3년 복 허용 주장, 사환국 정권 장악, 인조 반정 주도 세력, 정조 시대 탕평 정치 세력, 이이·성혼 문인 중심 학문 집단, 조선 후기 권력 구조 변화, 유학적 논의 기반 정당 형성


  1%|▏         | 1/73 [02:09<2:34:55, 129.10s/it]


🔎 [Retriever] 문제(generation-for-nlp-426) 분석 시작...
   🚦 [전략 선택]: INFERENCE
   ✨ [생성된 쿼리]: 의병계열, 애국계몽 운동 계열, 비밀결사, 총사령 박상진, 독립군 양성, 조직 결성


...


🔎 [Retriever] 문제(generation-for-nlp-512) 분석 시작...
   🚦 [전략 선택]: INFERENCE
   ✨ [생성된 쿼리]: 역사 탐정의 몽타주 검색어:  
백제사마왕, 영동대장군, 62세, 계묘년 5월 임진일(7일), 사년 8월 갑신일(12일), 안장, 대묘에 모시며 기록, 재위 기간 내 발생한 사건


 99%|█████████▊| 72/73 [58:03<01:08, 68.17s/it]


🔎 [Retriever] 문제(generation-for-nlp-513) 분석 시작...
   🚦 [전략 선택]: INFERENCE
   ✨ [생성된 쿼리]: 큰산과 깊은 골짜기가 많고 평원과 연못이 없음, 계곡을 따라 거주, 골짜기 물을 식수로 사용, 좋은 농경지 부족, 부지런히 농사 지어도 배부르지 않음, 성품 흉악하고 급함, 노략질 좋아함


100%|██████████| 73/73 [58:29<00:00, 48.08s/it]


🏆 최종 정답률: 50.68%


In [30]:
from google.colab import runtime
import time

print("✅ 모든 작업이 완료되었습니다!")
print("💾 데이터 저장이 잘 되었는지 마지막으로 확인하세요.")
print("👋 10초 뒤에 런타임을 종료하여 크레딧을 절약합니다...")

# 10초 카운트다운 (혹시라도 멈추고 싶으면 이때 중지 누르세요)
for i in range(10, 0, -1):
    print(f"{i}초...", end="\r")
    time.sleep(1)

print("\n🔌 런타임을 종료합니다. 수고하셨습니다!")
runtime.unassign()

✅ 모든 작업이 완료되었습니다!
💾 데이터 저장이 잘 되었는지 마지막으로 확인하세요.
👋 10초 뒤에 런타임을 종료하여 크레딧을 절약합니다...

🔌 런타임을 종료합니다. 수고하셨습니다!
